In [ ]:
import numpy as np
import pandas as pd
import neurokit2 as nk

import matplotlib.pyplot as plt

from keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score
import seaborn as sns
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, Flatten, Conv1D,TimeDistributed, Reshape, Activation, add, Layer, Concatenate, GRU, Bidirectional 
from tensorflow.keras.layers import MaxPooling1D, BatchNormalization, Dropout,Activation, GlobalAveragePooling1D

In [ ]:
# R-peak detection
def get_rpeak(signal, sampling_rate):
    _, rpeaks = nk.ecg_peaks(signal, sampling_rate=sampling_rate, show=False, method='neurokit')
    
    return rpeaks['ECG_R_Peaks']

# Segmentation Function

def load_BEAT_data(signal, r_peaks):
    signal_list = []
    for number in range(1,len(r_peaks)-1):
        signal_list.append(signal[r_peaks[number]-(int(left_sec*sampling_rate)) : r_peaks[number]+(int(right_sec*sampling_rate))])
    return np.array(signal_list)

def load_RHYTHM_data(data, rpeak, window):
    rhythm_data = []
    right = int(right_sec * sampling_rate)
    for i in range(1,len(rpeak)-1):
        if len(data[rpeak[i] + right : rpeak[i] + window + right]) == window:
            rhythm_data.append(data[rpeak[i] + right : rpeak[i]+right + window])
            
    return np.array(rhythm_data)

def load_WESAD(person, segment, window=0): 
    
    excitement_data, baseline_data, stress_data, meditation_data = [], [], [], []
    
    for i in person:
        df = pd.read_pickle(data_path + i + '.pkl')
        print(f"================== person = {i} ===========================")

        ex = df['excitement'].flatten()
        base = df['baseline'].flatten()
        stress = df['stress'].flatten()
        medi = df['meditation'].flatten()

        ex_r = get_rpeak(ex, sampling_rate)
        base_r = get_rpeak(base, sampling_rate)
        stress_r = get_rpeak(stress, sampling_rate)
        medi_r = get_rpeak(medi, sampling_rate)

        if segment == 'rhythm':
            excitement_data.append(load_RHYTHM_data(ex, ex_r, window)) 
            baseline_data.append(load_RHYTHM_data(base, base_r, window)) 
            stress_data.append(load_RHYTHM_data(stress, stress_r, window)) 
            meditation_data.append(load_RHYTHM_data(medi, medi_r, window)) 
        else :
            excitement_data.append(load_BEAT_data(ex, ex_r)) 
            baseline_data.append(load_BEAT_data(base, base_r)) 
            stress_data.append(load_BEAT_data(stress, stress_r)) 
            meditation_data.append(load_BEAT_data(medi, medi_r)) 
            
    Amuse = np.vstack(excitement_data)
    Base = np.vstack(baseline_data)
    Stress = np.vstack(stress_data)
    Meditation = np.vstack(meditation_data)
    
    amuse_label = np.zeros(Amuse.shape[0], np.int32)
    normal_label = np.ones(Base.shape[0], np.int32)
    stress_label = 2*np.ones(Stress.shape[0], np.int32)
    meditation_label = 3*np.ones(Meditation.shape[0], np.int32)
    
    Train = np.concatenate([Amuse, Base, Stress, Meditation])
    Train_label = np.concatenate([amuse_label, normal_label, stress_label, meditation_label])
    
    return Train, Train_label

def shuffle_data2(data1, data2, labels):
    # data1, data2, label을 입력받아 무작위로 섞고 (인덱스 위치는 동일하게) Return
    indices = np.arange(data1.shape[0])
    np.random.shuffle(indices)
    return data1[indices], data2[indices], labels[indices]


def Plot_lr_curve(history):
    
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy'] 
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    
    plt.figure(figsize=(20,5))
    plt.subplot(121)
    plt.plot(acc, label='Training Accuracy')
    plt.plot(val_acc, label='Validation Accuracy')
    plt.legend()
    plt.ylabel('Accuracy')
    plt.title('Training and Validation Accuracy')

    plt.subplot(122)
    plt.plot(loss, label='Training Loss')
    plt.plot(val_loss, label='Validation Loss')
    plt.legend()
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('epoch')

def Plot_confusion_matrix(true_d, pred, class_name):
    
    pred_label = []
    true_label = []
    
    for i in range(len(pred)):
        pred_label.append(np.argmax(pred[i]))
        
    for i in range(len(true_d)):
        true_label.append(np.argmax(true_d[i]))
    
    print(pred_label[0:10])
    # Accuracy
    accuracy = accuracy_score(true_label, pred_label)
    print('accuracy : ', np.round(accuracy,3))
    
    # Precision
    precision = precision_score(true_label, pred_label, average='macro')
    print('precision : ', np.round(precision,3))
    
    # Recall
    recall = recall_score(true_label, pred_label,average='macro')
    print('recall : ', np.round(recall,3))
    
    # F1 Score
    F1_score = f1_score(true_label, pred_label,average='macro')
    print('F1_score : ', np.round(F1_score,3))

    #clasification report
    report = classification_report(true_label, pred_label, target_names=class_name)#'Amuse',
    print(report)
    
    #confusion matrix
    confusion = confusion_matrix(true_label, pred_label)
    disp = ConfusionMatrixDisplay(confusion, display_labels=class_name)#'Amuse',
    disp.plot(cmap=plt.cm.Blues)
    plt.show()
    
    #plt.figure(figsize=(10,5))
    cmn = confusion.astype('float') / confusion.sum(axis=1)[:, np.newaxis]
    sns.heatmap(cmn, annot=True, fmt='.2f', xticklabels=class_name, yticklabels=class_name)
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.show(block=False)

In [ ]:
def load_data(person, beat, rhythm, window):
        rhythm_train, rhythm_label = load_WESAD(person, 'rhythm', window)
        beat_train, beat_label = load_WESAD(person, 'beat', window)
        
        beat_train = beat_train[:rhythm_train.shape[0]]
        beat_label = beat_label[:rhythm_label.shape[0]]
        
        train_beat, train_rhythm, train_y = shuffle_data2(beat_train, rhythm_train, beat_label)
        
        return train_beat, train_rhythm, train_y

In [ ]:
data_path = "D:\Journal\WESAD_Denoise_Outlier_Normalization_DB/"
data_path = data_path.replace('\\', '/')

np.random.seed(128)
random_state = 128

sampling_rate = 700
n_split = 5
left_sec = 0.24
right_sec = 0.4

person = np.array(['S2','S3','S4','S5','S6','S7','S8','S9','S10','S11','S13','S14','S15','S16','S17'])
class_name = np.array(['Amusement', 'Baseline', 'Stress','Meditation'])

In [ ]:
window_list = [i*sampling_rate for i in range(2,11)]
print(window_list)

In [ ]:
train_beat, train_rhythm, train_y = load_data(person, 'beat', 'rhythm', sampling_rate*8)

## Beat network 실험

## Model

In [ ]:
from  tensorflow.keras.initializers import HeNormal
from keras.callbacks import EarlyStopping


def scheuler(epoch, lr):
    if epoch < 41:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

callback_lr = tf.keras.callbacks.LearningRateScheduler(scheuler)



# 조기 종료 
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, min_delta=0.005, restore_best_weights=True)

# 가중치 초기화 
weight_init = HeNormal(seed= 128)



In [ ]:
def beat_model(re_X_train_b):
    input_beat = Input(shape=(re_X_train_b,1), name = 'input_beat')

    conv1_B = Conv1D(8, 5, activation=None, padding='same', kernel_initializer=weight_init)(input_beat)
    conv2_B = Conv1D(8, 5, activation=None, padding='same', kernel_initializer=weight_init)(conv1_B)
    bn1_B = BatchNormalization()(conv2_B)
    act_B = Activation('relu')(bn1_B)
    max1_B = MaxPooling1D(2,2)(act_B)

    conv1_B_1 = Conv1D(16, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_B)
    conv2_B_1 = Conv1D(16, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_B_1)
    bn1_B_1 = BatchNormalization()(conv2_B_1)
    act_B_1 = Activation('relu')(bn1_B_1)
    max1_B_1 = MaxPooling1D(2,2)(act_B_1)


    conv1_B_2 = Conv1D(32, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_B_1)
    conv2_B_2 = Conv1D(32, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_B_2)
    bn1_B_2 = BatchNormalization()(conv2_B_2)
    act_B_2 = Activation('relu')(bn1_B_2)
    max1_B_2 = MaxPooling1D(2,2)(act_B_2)

    conv1_B_3 = Conv1D(64, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_B_2)
    conv2_B_3 = Conv1D(64, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_B_3)
    bn1_B_3 = BatchNormalization()(conv2_B_3)
    act_B_3 = Activation('relu')(bn1_B_3)
    max1_B_3 = MaxPooling1D(2,2)(act_B_3)

    conv1_B_4 = Conv1D(128, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_B_3)
    conv2_B_4 = Conv1D(128, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_B_4)
    bn1_B_4 = BatchNormalization()(conv2_B_4)
    act_B_4 = Activation('relu')(bn1_B_4)
    max1_B_4 = MaxPooling1D(2,2)(act_B_4)

    # conv1_B_5 = Conv1D(256, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_B_4)
    # conv2_B_5 = Conv1D(256, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_B_5)
    # bn1_B_5 = BatchNormalization()(conv2_B_5)
    # act_B_5 = Activation('relu')(bn1_B_5)
    # max1_B_5 = MaxPooling1D(2,2)(act_B_5)

    # conv1_B_6 = Conv1D(512, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_B_5)
    # conv2_B_6 = Conv1D(512, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_B_6)
    # bn1_B_6 = BatchNormalization()(conv2_B_6)
    # act_B_6 = Activation('relu')(bn1_B_6)
    # max1_B_6 = MaxPooling1D(2,2)(act_B_6)

    # conv1_B_7 = Conv1D(256, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_B_6)
    # conv2_B_7 = Conv1D(256, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_B_7)
    # bn1_B_7 = BatchNormalization()(conv2_B_7)
    # act_B_7 = Activation('relu')(bn1_B_7)
    # max1_B_7 = MaxPooling1D(2,2)(act_B_7)

    GAP_B = GlobalAveragePooling1D()(max1_B_4)

    re_R = Reshape((1, GAP_B.shape[1]))(GAP_B)
    x_R_LSTM = LSTM(128,return_sequences=True)(re_R)
    x_R_LSTM2 = LSTM(64, return_sequences=True)(x_R_LSTM)
    fl_f = Flatten()(x_R_LSTM2)

    den1 = Dense(128, activation='relu')(fl_f)
    den2 = Dense(64, activation='relu')(den1)

    fusion_output = Dense(4, activation='softmax')(den2)

    fusion_model = Model(inputs=input_beat, outputs= fusion_output)
    fusion_model.summary()
    
    return fusion_model

In [ ]:
#counts = np.sum(train, axis=0)
counts = np.bincount(train_y)
print(counts)

amuse_num = counts[0]
normal_num = counts[1]
stress_num = counts[2]
meditation_num = counts[3]
total = amuse_num + normal_num + stress_num + meditation_num

weight_amuse = (1/amuse_num) * (total/4.0)
weight_normal = (1/normal_num) * (total/4.0)
weight_stress = (1/stress_num) * (total/4.0)
weight_meditation  = (1/meditation_num) * (total/4.0)
class_weight = {0:weight_amuse, 1:weight_normal, 2:weight_stress, 3:weight_meditation}
print(total, class_weight)


In [ ]:
from sklearn.model_selection import train_test_split

train_b, test_b, train_label, test_label = train_test_split(train_beat, train_y,
                                                            test_size=0.2,
                                                            stratify=train_y,
                                                            random_state=128,
                                                            shuffle=True)

train_b, val_b, train_label, val_label = train_test_split(train_b, train_label,
                                                         test_size=0.25,
                                                         stratify=train_label,
                                                         random_state=128,
                                                         shuffle=True)

print(train_b.shape, val_b.shape, test_b.shape)

In [ ]:
train_label = to_categorical(train_label)
val_label = to_categorical(val_label)
test_label = to_categorical(test_label)

In [ ]:
model_b = beat_model(train_b.shape[1])
callback_lr = tf.keras.callbacks.LearningRateScheduler(scheuler)
model_b.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

early_stopping
history = model_b.fit(train_b, train_label,
                      validation_data=(val_b, val_label),
                      epochs=200,
                      batch_size=32,
                      callbacks=[early_stopping],
                      class_weight = class_weight
                     )

In [ ]:
scores = model_b.evaluate(test_b, test_label)
Plot_lr_curve(history)
pred = model_b.predict(test_b)
Plot_confusion_matrix(test_label, pred)

# Beat 학습률 0.0001 실험

In [ ]:
from sklearn.model_selection import StratifiedKFold
def cal_class_weight(label):
    
    counts = np.bincount(label)
    
    amuse_num = counts[0]
    normal_num = counts[1]
    stress_num = counts[2]
    meditation_num = counts[3]
    total = amuse_num + normal_num + stress_num + meditation_num
    
    weight_amuse = (1/amuse_num) * (total/4.0)
    weight_normal = (1/normal_num) * (total/4.0)
    weight_stress = (1/stress_num) * (total/4.0)
    weight_meditation  = (1/meditation_num) * (total/4.0)
    class_weight = {0:weight_amuse, 1:weight_normal, 2:weight_stress, 3:weight_meditation}
    # print(total, class_weight)

    return class_weight

In [ ]:
train_beat, train_rhythm, train_y = load_data(person, 'beat', 'rhythm', sampling_rate*8)

In [ ]:
## layer 실험
number = 1
save_model_path = "D:/Journal/WESAD_Model/Beat_Rhythm_5Fold_실험/"
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=128)

for train_index, test_index in skf.split(train_beat, train_y):
    X_train_b, X_test_b = train_beat[train_index], train_beat[test_index]
    Y_train, Y_test = train_y[train_index], train_y[test_index]

    y_train = to_categorical(Y_train)
    y_test = to_categorical(Y_test)
    class_weight = cal_class_weight(Y_train)

    MAUS_beat_model = beat_model(X_train_b.shape[1])
    callback_lr = tf.keras.callbacks.LearningRateScheduler(scheuler)
    MAUS_beat_model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    history = MAUS_beat_model.fit(X_train_b, y_train, 
                        validation_data=(X_test_b, y_test), 
                        epochs=50, 
                        batch_size=32,
                        callbacks=[callback_lr],
                        class_weight=class_weight
                       ) 
    
    
    scores = MAUS_beat_model.evaluate(X_test_b, y_test)
    Plot_lr_curve(history)
    pred = MAUS_beat_model.predict(X_test_b)
    Plot_confusion_matrix(y_test, pred, class_name)
    
    
    MAUS_beat_model.save(save_model_path + 'WESAD_beat_8020_(0.0001)_Fold_'  +str(number)+ '(' + str(np.round(scores[1]*100,2)) + ').h5')


## Rhythm network 실험

In [ ]:
from  tensorflow.keras.initializers import HeNormal
from keras.callbacks import EarlyStopping


def scheuler(epoch, lr):
    if epoch < 41:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

callback_lr = tf.keras.callbacks.LearningRateScheduler(scheuler)



# 조기 종료 
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, min_delta=0.005, restore_best_weights=True)

# 가중치 초기화 
weight_init = HeNormal(seed= 128)

#counts = np.sum(train, axis=0)

def cal_class_weight(label):
    
    counts = np.bincount(label)
    
    amuse_num = counts[0]
    normal_num = counts[1]
    stress_num = counts[2]
    meditation_num = counts[3]
    total = amuse_num + normal_num + stress_num + meditation_num
    
    weight_amuse = (1/amuse_num) * (total/4.0)
    weight_normal = (1/normal_num) * (total/4.0)
    weight_stress = (1/stress_num) * (total/4.0)
    weight_meditation  = (1/meditation_num) * (total/4.0)
    class_weight = {0:weight_amuse, 1:weight_normal, 2:weight_stress, 3:weight_meditation}
    # print(total, class_weight)

    return class_weight


In [ ]:
def shuffle_data(data, labels):
    indices = np.arange(data.shape[0])
    np.random.shuffle(indices)
    return data[indices], labels[indices]

def load_data(person, rhythm, window):
        rhythm_train, rhythm_label = load_WESAD(person, 'rhythm', window)
        train_rhythm, train_y = shuffle_data(rhythm_train, rhythm_label)
        
        return train_rhythm, train_y

In [ ]:
def rhythm_model(re_X_train_r):
    input_rhythm = Input(shape=(re_X_train_r,1), name = 'input_rhythm')

    conv1_R = Conv1D(8, 5, activation=None, padding='same', kernel_initializer=weight_init)(input_rhythm)
    conv2_R = Conv1D(8, 5, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R)
    bn1_R = BatchNormalization()(conv2_R)
    act_R = Activation('relu')(bn1_R)
    max1_R = MaxPooling1D(2,2)(act_R)

    conv1_R_1 = Conv1D(16, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_R)
    conv2_R_1 = Conv1D(16, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R_1)
    bn1_R_1 = BatchNormalization()(conv2_R_1)
    act_R_1 = Activation('relu')(bn1_R_1)
    max1_R_1 = MaxPooling1D(2,2)(act_R_1)


    conv1_R_2 = Conv1D(32, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_R_1)
    conv2_R_2 = Conv1D(32, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R_2)
    bn1_R_2 = BatchNormalization()(conv2_R_2)
    act_R_2 = Activation('relu')(bn1_R_2)
    max1_R_2 = MaxPooling1D(2,2)(act_R_2)


    conv1_R_3 = Conv1D(64, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_R_2)
    conv2_R_3 = Conv1D(64, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R_3)
    bn1_R_3 = BatchNormalization()(conv2_R_3)
    act_R_3 = Activation('relu')(bn1_R_3)
    max1_R_3 = MaxPooling1D(2,2)(act_R_3)


    # conv1_R_4 = Conv1D(128, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_R_3)
    # conv2_R_4 = Conv1D(128, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R_4)
    # bn1_R_4 = BatchNormalization()(conv2_R_4)
    # act_R_4 = Activation('relu')(bn1_R_4)
    # max1_R_4 = MaxPooling1D(2,2)(act_R_4)

    # conv1_R_5 = Conv1D(256, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_R_4)
    # conv2_R_5 = Conv1D(256, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R_5)
    # bn1_R_5 = BatchNormalization()(conv2_R_5)
    # act_R_5 = Activation('relu')(bn1_R_5)
    # max1_R_5 = MaxPooling1D(2,2)(act_R_5)

    # conv1_R_6 = Conv1D(512, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_R_5)
    # conv2_R_6 = Conv1D(512, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R_6)
    # bn1_R_6 = BatchNormalization()(conv2_R_6)
    # act_R_6 = Activation('relu')(bn1_R_6)
    # max1_R_6 = MaxPooling1D(2,2)(act_R_6)

    # conv1_R_7 = Conv1D(256, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_R_6)
    # conv2_R_7 = Conv1D(256, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R_7)
    # bn1_R_7 = BatchNormalization()(conv2_R_7)
    # act_R_7 = Activation('relu')(bn1_R_7)
    # max1_R_7 = MaxPooling1D(2,2)(act_R_7)

    # conv1_R_8 = Conv1D(128, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_R_7)
    # conv2_R_8 = Conv1D(128, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R_8)
    # bn1_R_8 = BatchNormalization()(conv2_R_8)
    # act_R_8 = Activation('relu')(bn1_R_8)
    # max1_R_8 = MaxPooling1D(2,2)(act_R_8)

    GAP_R = GlobalAveragePooling1D()(max1_R_2)

    re_R = Reshape((1, GAP_R.shape[1]))(GAP_R)
    x_R_LSTM = LSTM(128,return_sequences=True)(re_R)
    x_R_LSTM2 = LSTM(64, return_sequences=True)(x_R_LSTM)
    fl_f = Flatten()(x_R_LSTM2)

    den1 = Dense(128, activation='relu')(fl_f)
    den2 = Dense(64, activation='relu')(den1)

    fusion_output = Dense(4, activation='softmax')(den2)

    fusion_model = Model(inputs= input_rhythm, outputs= fusion_output)
    fusion_model.summary()
    
    return fusion_model

In [ ]:
save_model_path = "D:/Journal/WESAD_Model/Rhythm_layer_setting/"

## layer 1

In [ ]:
for window in window_list:

    print("======================================" + str(window) + "======================================")
    
    train_rhythm, train_y = load_data(person, 'rhythm', window)

    train_r, test_r, train_label, test_label = train_test_split(train_rhythm, train_y,
                                                                test_size=0.2,
                                                                stratify=train_y,
                                                                random_state=128,
                                                                shuffle=True)
    
    train_r, val_r, train_label, val_label = train_test_split(train_r, train_label,
                                                             test_size=0.25,
                                                             stratify=train_label,
                                                             random_state=128,
                                                             shuffle=True)
    print(train_r.shape, val_r.shape, test_r.shape)
    
    class_weight = cal_class_weight(train_label)
    train_label = to_categorical(train_label)
    val_label = to_categorical(val_label)
    test_label = to_categorical(test_label)

    model_r = rhythm_model(train_r.shape[1])
    model_r.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    
    history = model_r.fit(train_r, train_label,
                          validation_data=(val_r, val_label),
                          epochs=200,
                          batch_size=32,
                          callbacks=[early_stopping],
                          class_weight = class_weight
                         )

    scores = model_r.evaluate(test_r, test_label)
    Plot_lr_curve(history)
    pred = model_r.predict(test_r)
    Plot_confusion_matrix(test_label, pred)
    
    model_r.save(save_model_path + 'wesad_rhythm_' + str(window) + 'layer1_'+ '6,2,2' + '(' + str(np.round(scores[1]*100,2)) + ')_.h5')

#  layer 2

In [ ]:

for window in window_list:

    print("======================================" + str(window) + "======================================")
    
    train_rhythm, train_y = load_data(person, 'rhythm', window)

    train_r, test_r, train_label, test_label = train_test_split(train_rhythm, train_y,
                                                                test_size=0.2,
                                                                stratify=train_y,
                                                                random_state=128,
                                                                shuffle=True)
    
    train_r, val_r, train_label, val_label = train_test_split(train_r, train_label,
                                                             test_size=0.25,
                                                             stratify=train_label,
                                                             random_state=128,
                                                             shuffle=True)
    print(train_r.shape, val_r.shape, test_r.shape)
    
    class_weight = cal_class_weight(train_label)
    train_label = to_categorical(train_label)
    val_label = to_categorical(val_label)
    test_label = to_categorical(test_label)

    model_r = rhythm_model(train_r.shape[1])
    model_r.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    
    history = model_r.fit(train_r, train_label,
                          validation_data=(val_r, val_label),
                          epochs=200,
                          batch_size=32,
                          callbacks=[early_stopping],
                          class_weight = class_weight
                         )

    scores = model_r.evaluate(test_r, test_label)
    Plot_lr_curve(history)
    pred = model_r.predict(test_r)
    Plot_confusion_matrix(test_label, pred)
    
    model_r.save(save_model_path + 'wesad_rhythm_' + str(window) + 'layer2_'+ '6,2,2' + '(' + str(np.round(scores[1]*100,2)) + ')_.h5')

# layer 3

In [ ]:

for window in window_list:

    print("======================================" + str(window) + "======================================")
    
    train_rhythm, train_y = load_data(person, 'rhythm', window)

    train_r, test_r, train_label, test_label = train_test_split(train_rhythm, train_y,
                                                                test_size=0.2,
                                                                stratify=train_y,
                                                                random_state=128,
                                                                shuffle=True)
    
    train_r, val_r, train_label, val_label = train_test_split(train_r, train_label,
                                                             test_size=0.25,
                                                             stratify=train_label,
                                                             random_state=128,
                                                             shuffle=True)
    print(train_r.shape, val_r.shape, test_r.shape)
    
    class_weight = cal_class_weight(train_label)
    train_label = to_categorical(train_label)
    val_label = to_categorical(val_label)
    test_label = to_categorical(test_label)

    model_r = rhythm_model(train_r.shape[1])
    model_r.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    
    history = model_r.fit(train_r, train_label,
                          validation_data=(val_r, val_label),
                          epochs=200,
                          batch_size=32,
                          callbacks=[early_stopping],
                          class_weight = class_weight
                         )

    scores = model_r.evaluate(test_r, test_label)
    Plot_lr_curve(history)
    pred = model_r.predict(test_r)
    Plot_confusion_matrix(test_label, pred)
    
    model_r.save(save_model_path + 'wesad_rhythm_' + str(window) + 'layer3_'+ '6,2,2' + '(' + str(np.round(scores[1]*100,2)) + ')_.h5')

# layer 4

In [ ]:
def rhythm_model(re_X_train_r):
    input_rhythm = Input(shape=(re_X_train_r,1), name = 'input_rhythm')

    conv1_R = Conv1D(8, 5, activation=None, padding='same', kernel_initializer=weight_init)(input_rhythm)
    conv2_R = Conv1D(8, 5, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R)
    bn1_R = BatchNormalization()(conv2_R)
    act_R = Activation('relu')(bn1_R)
    max1_R = MaxPooling1D(2,2)(act_R)

    conv1_R_1 = Conv1D(16, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_R)
    conv2_R_1 = Conv1D(16, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R_1)
    bn1_R_1 = BatchNormalization()(conv2_R_1)
    act_R_1 = Activation('relu')(bn1_R_1)
    max1_R_1 = MaxPooling1D(2,2)(act_R_1)


    conv1_R_2 = Conv1D(32, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_R_1)
    conv2_R_2 = Conv1D(32, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R_2)
    bn1_R_2 = BatchNormalization()(conv2_R_2)
    act_R_2 = Activation('relu')(bn1_R_2)
    max1_R_2 = MaxPooling1D(2,2)(act_R_2)


    conv1_R_3 = Conv1D(64, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_R_2)
    conv2_R_3 = Conv1D(64, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R_3)
    bn1_R_3 = BatchNormalization()(conv2_R_3)
    act_R_3 = Activation('relu')(bn1_R_3)
    max1_R_3 = MaxPooling1D(2,2)(act_R_3)


    GAP_R = GlobalAveragePooling1D()(max1_R_3)

    re_R = Reshape((1, GAP_R.shape[1]))(GAP_R)
    x_R_LSTM = LSTM(128,return_sequences=True)(re_R)
    x_R_LSTM2 = LSTM(64, return_sequences=True)(x_R_LSTM)
    fl_f = Flatten()(x_R_LSTM2)

    den1 = Dense(128, activation='relu')(fl_f)
    den2 = Dense(64, activation='relu')(den1)

    fusion_output = Dense(4, activation='softmax')(den2)

    fusion_model = Model(inputs= input_rhythm, outputs= fusion_output)
    fusion_model.summary()
    
    return fusion_model

In [ ]:
for window in window_list:

    print("======================================" + str(window) + "======================================")
    
    train_rhythm, train_y = load_data(person, 'rhythm', window)

    train_r, test_r, train_label, test_label = train_test_split(train_rhythm, train_y,
                                                                test_size=0.2,
                                                                stratify=train_y,
                                                                random_state=128,
                                                                shuffle=True)
    
    train_r, val_r, train_label, val_label = train_test_split(train_r, train_label,
                                                             test_size=0.25,
                                                             stratify=train_label,
                                                             random_state=128,
                                                             shuffle=True)
    print(train_r.shape, val_r.shape, test_r.shape)
    
    class_weight = cal_class_weight(train_label)
    train_label = to_categorical(train_label)
    val_label = to_categorical(val_label)
    test_label = to_categorical(test_label)

    model_r = rhythm_model(train_r.shape[1])
    model_r.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    
    history = model_r.fit(train_r, train_label,
                          validation_data=(val_r, val_label),
                          epochs=200,
                          batch_size=32,
                          callbacks=[early_stopping],
                          class_weight = class_weight
                         )

    scores = model_r.evaluate(test_r, test_label)
    Plot_lr_curve(history)
    pred = model_r.predict(test_r)
    Plot_confusion_matrix(test_label, pred)
    
    model_r.save(save_model_path + 'wesad_rhythm_' + str(window) + 'layer4_'+ '6,2,2' + '(' + str(np.round(scores[1]*100,2)) + ')_.h5')

# layer 5

In [ ]:
def rhythm_model(re_X_train_r):
    input_rhythm = Input(shape=(re_X_train_r,1), name = 'input_rhythm')

    conv1_R = Conv1D(8, 5, activation=None, padding='same', kernel_initializer=weight_init)(input_rhythm)
    conv2_R = Conv1D(8, 5, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R)
    bn1_R = BatchNormalization()(conv2_R)
    act_R = Activation('relu')(bn1_R)
    max1_R = MaxPooling1D(2,2)(act_R)

    conv1_R_1 = Conv1D(16, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_R)
    conv2_R_1 = Conv1D(16, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R_1)
    bn1_R_1 = BatchNormalization()(conv2_R_1)
    act_R_1 = Activation('relu')(bn1_R_1)
    max1_R_1 = MaxPooling1D(2,2)(act_R_1)


    conv1_R_2 = Conv1D(32, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_R_1)
    conv2_R_2 = Conv1D(32, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R_2)
    bn1_R_2 = BatchNormalization()(conv2_R_2)
    act_R_2 = Activation('relu')(bn1_R_2)
    max1_R_2 = MaxPooling1D(2,2)(act_R_2)


    conv1_R_3 = Conv1D(64, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_R_2)
    conv2_R_3 = Conv1D(64, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R_3)
    bn1_R_3 = BatchNormalization()(conv2_R_3)
    act_R_3 = Activation('relu')(bn1_R_3)
    max1_R_3 = MaxPooling1D(2,2)(act_R_3)


    conv1_R_4 = Conv1D(128, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_R_3)
    conv2_R_4 = Conv1D(128, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R_4)
    bn1_R_4 = BatchNormalization()(conv2_R_4)
    act_R_4 = Activation('relu')(bn1_R_4)
    max1_R_4 = MaxPooling1D(2,2)(act_R_4)


    GAP_R = GlobalAveragePooling1D()(max1_R_4)

    re_R = Reshape((1, GAP_R.shape[1]))(GAP_R)
    x_R_LSTM = LSTM(128,return_sequences=True)(re_R)
    x_R_LSTM2 = LSTM(64, return_sequences=True)(x_R_LSTM)
    fl_f = Flatten()(x_R_LSTM2)

    den1 = Dense(128, activation='relu')(fl_f)
    den2 = Dense(64, activation='relu')(den1)

    fusion_output = Dense(4, activation='softmax')(den2)

    fusion_model = Model(inputs= input_rhythm, outputs= fusion_output)
    fusion_model.summary()
    
    return fusion_model

In [ ]:

for window in window_list:

    print("======================================" + str(window) + "======================================")
    
    train_rhythm, train_y = load_data(person, 'rhythm', window)

    train_r, test_r, train_label, test_label = train_test_split(train_rhythm, train_y,
                                                                test_size=0.2,
                                                                stratify=train_y,
                                                                random_state=128,
                                                                shuffle=True)
    
    train_r, val_r, train_label, val_label = train_test_split(train_r, train_label,
                                                             test_size=0.25,
                                                             stratify=train_label,
                                                             random_state=128,
                                                             shuffle=True)
    print(train_r.shape, val_r.shape, test_r.shape)
    
    class_weight = cal_class_weight(train_label)
    train_label = to_categorical(train_label)
    val_label = to_categorical(val_label)
    test_label = to_categorical(test_label)

    model_r = rhythm_model(train_r.shape[1])
    model_r.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    
    history = model_r.fit(train_r, train_label,
                          validation_data=(val_r, val_label),
                          epochs=200,
                          batch_size=32,
                          callbacks=[early_stopping],
                          class_weight = class_weight
                         )

    scores = model_r.evaluate(test_r, test_label)
    Plot_lr_curve(history)
    pred = model_r.predict(test_r)
    Plot_confusion_matrix(test_label, pred)
    
    model_r.save(save_model_path + 'wesad_rhythm_' + str(window) + 'layer5_'+ '6,2,2' + '(' + str(np.round(scores[1]*100,2)) + ')_.h5')

# layer 6

In [ ]:
def rhythm_model(re_X_train_r):
    input_rhythm = Input(shape=(re_X_train_r,1), name = 'input_rhythm')

    conv1_R = Conv1D(8, 5, activation=None, padding='same', kernel_initializer=weight_init)(input_rhythm)
    conv2_R = Conv1D(8, 5, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R)
    bn1_R = BatchNormalization()(conv2_R)
    act_R = Activation('relu')(bn1_R)
    max1_R = MaxPooling1D(2,2)(act_R)

    conv1_R_1 = Conv1D(16, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_R)
    conv2_R_1 = Conv1D(16, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R_1)
    bn1_R_1 = BatchNormalization()(conv2_R_1)
    act_R_1 = Activation('relu')(bn1_R_1)
    max1_R_1 = MaxPooling1D(2,2)(act_R_1)


    conv1_R_2 = Conv1D(32, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_R_1)
    conv2_R_2 = Conv1D(32, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R_2)
    bn1_R_2 = BatchNormalization()(conv2_R_2)
    act_R_2 = Activation('relu')(bn1_R_2)
    max1_R_2 = MaxPooling1D(2,2)(act_R_2)


    conv1_R_3 = Conv1D(64, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_R_2)
    conv2_R_3 = Conv1D(64, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R_3)
    bn1_R_3 = BatchNormalization()(conv2_R_3)
    act_R_3 = Activation('relu')(bn1_R_3)
    max1_R_3 = MaxPooling1D(2,2)(act_R_3)


    conv1_R_4 = Conv1D(128, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_R_3)
    conv2_R_4 = Conv1D(128, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R_4)
    bn1_R_4 = BatchNormalization()(conv2_R_4)
    act_R_4 = Activation('relu')(bn1_R_4)
    max1_R_4 = MaxPooling1D(2,2)(act_R_4)

    conv1_R_5 = Conv1D(256, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_R_4)
    conv2_R_5 = Conv1D(256, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R_5)
    bn1_R_5 = BatchNormalization()(conv2_R_5)
    act_R_5 = Activation('relu')(bn1_R_5)
    max1_R_5 = MaxPooling1D(2,2)(act_R_5)

    GAP_R = GlobalAveragePooling1D()(max1_R_5)

    re_R = Reshape((1, GAP_R.shape[1]))(GAP_R)
    x_R_LSTM = LSTM(128,return_sequences=True)(re_R)
    x_R_LSTM2 = LSTM(64, return_sequences=True)(x_R_LSTM)
    fl_f = Flatten()(x_R_LSTM2)

    den1 = Dense(128, activation='relu')(fl_f)
    den2 = Dense(64, activation='relu')(den1)

    fusion_output = Dense(4, activation='softmax')(den2)

    fusion_model = Model(inputs= input_rhythm, outputs= fusion_output)
    fusion_model.summary()
    
    return fusion_model

In [ ]:

for window in window_list:

    print("======================================" + str(window) + "======================================")
    
    train_rhythm, train_y = load_data(person, 'rhythm', window)

    train_r, test_r, train_label, test_label = train_test_split(train_rhythm, train_y,
                                                                test_size=0.2,
                                                                stratify=train_y,
                                                                random_state=128,
                                                                shuffle=True)
    
    train_r, val_r, train_label, val_label = train_test_split(train_r, train_label,
                                                             test_size=0.25,
                                                             stratify=train_label,
                                                             random_state=128,
                                                             shuffle=True)
    print(train_r.shape, val_r.shape, test_r.shape)
    
    class_weight = cal_class_weight(train_label)
    train_label = to_categorical(train_label)
    val_label = to_categorical(val_label)
    test_label = to_categorical(test_label)

    model_r = rhythm_model(train_r.shape[1])
    model_r.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    
    history = model_r.fit(train_r, train_label,
                          validation_data=(val_r, val_label),
                          epochs=200,
                          batch_size=32,
                          callbacks=[early_stopping],
                          class_weight = class_weight
                         )

    scores = model_r.evaluate(test_r, test_label)
    Plot_lr_curve(history)
    pred = model_r.predict(test_r)
    Plot_confusion_matrix(test_label, pred)
    
    model_r.save(save_model_path + 'wesad_rhythm_' + str(window) + 'layer6_'+ '6,2,2' + '(' + str(np.round(scores[1]*100,2)) + ')_.h5')

# layer 7

In [ ]:
def rhythm_model(re_X_train_r):
    input_rhythm = Input(shape=(re_X_train_r,1), name = 'input_rhythm')

    conv1_R = Conv1D(8, 5, activation=None, padding='same', kernel_initializer=weight_init)(input_rhythm)
    conv2_R = Conv1D(8, 5, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R)
    bn1_R = BatchNormalization()(conv2_R)
    act_R = Activation('relu')(bn1_R)
    max1_R = MaxPooling1D(2,2)(act_R)

    conv1_R_1 = Conv1D(16, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_R)
    conv2_R_1 = Conv1D(16, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R_1)
    bn1_R_1 = BatchNormalization()(conv2_R_1)
    act_R_1 = Activation('relu')(bn1_R_1)
    max1_R_1 = MaxPooling1D(2,2)(act_R_1)


    conv1_R_2 = Conv1D(32, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_R_1)
    conv2_R_2 = Conv1D(32, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R_2)
    bn1_R_2 = BatchNormalization()(conv2_R_2)
    act_R_2 = Activation('relu')(bn1_R_2)
    max1_R_2 = MaxPooling1D(2,2)(act_R_2)


    conv1_R_3 = Conv1D(64, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_R_2)
    conv2_R_3 = Conv1D(64, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R_3)
    bn1_R_3 = BatchNormalization()(conv2_R_3)
    act_R_3 = Activation('relu')(bn1_R_3)
    max1_R_3 = MaxPooling1D(2,2)(act_R_3)


    conv1_R_4 = Conv1D(128, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_R_3)
    conv2_R_4 = Conv1D(128, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R_4)
    bn1_R_4 = BatchNormalization()(conv2_R_4)
    act_R_4 = Activation('relu')(bn1_R_4)
    max1_R_4 = MaxPooling1D(2,2)(act_R_4)

    conv1_R_5 = Conv1D(256, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_R_4)
    conv2_R_5 = Conv1D(256, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R_5)
    bn1_R_5 = BatchNormalization()(conv2_R_5)
    act_R_5 = Activation('relu')(bn1_R_5)
    max1_R_5 = MaxPooling1D(2,2)(act_R_5)

    conv1_R_6 = Conv1D(512, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_R_5)
    conv2_R_6 = Conv1D(512, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R_6)
    bn1_R_6 = BatchNormalization()(conv2_R_6)
    act_R_6 = Activation('relu')(bn1_R_6)
    max1_R_6 = MaxPooling1D(2,2)(act_R_6)

    GAP_R = GlobalAveragePooling1D()(max1_R_6)

    re_R = Reshape((1, GAP_R.shape[1]))(GAP_R)
    x_R_LSTM = LSTM(128,return_sequences=True)(re_R)
    x_R_LSTM2 = LSTM(64, return_sequences=True)(x_R_LSTM)
    fl_f = Flatten()(x_R_LSTM2)

    den1 = Dense(128, activation='relu')(fl_f)
    den2 = Dense(64, activation='relu')(den1)

    fusion_output = Dense(4, activation='softmax')(den2)

    fusion_model = Model(inputs= input_rhythm, outputs= fusion_output)
    fusion_model.summary()
    
    return fusion_model

In [ ]:

for window in window_list:

    print("======================================" + str(window) + "======================================")
    
    train_rhythm, train_y = load_data(person, 'rhythm', window)

    train_r, test_r, train_label, test_label = train_test_split(train_rhythm, train_y,
                                                                test_size=0.2,
                                                                stratify=train_y,
                                                                random_state=128,
                                                                shuffle=True)
    
    train_r, val_r, train_label, val_label = train_test_split(train_r, train_label,
                                                             test_size=0.25,
                                                             stratify=train_label,
                                                             random_state=128,
                                                             shuffle=True)
    print(train_r.shape, val_r.shape, test_r.shape)
    
    class_weight = cal_class_weight(train_label)
    train_label = to_categorical(train_label)
    val_label = to_categorical(val_label)
    test_label = to_categorical(test_label)

    model_r = rhythm_model(train_r.shape[1])
    model_r.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    
    history = model_r.fit(train_r, train_label,
                          validation_data=(val_r, val_label),
                          epochs=200,
                          batch_size=32,
                          callbacks=[early_stopping],
                          class_weight = class_weight
                         )

    scores = model_r.evaluate(test_r, test_label)
    Plot_lr_curve(history)
    pred = model_r.predict(test_r)
    Plot_confusion_matrix(test_label, pred)
    
    model_r.save(save_model_path + 'wesad_rhythm_' + str(window) + 'layer7_'+ '6,2,2' + '(' + str(np.round(scores[1]*100,2)) + ')_.h5')

# Rhythm 5-Fold

# 5 layer and 3 second

In [ ]:
train_beat, train_rhythm, train_y = load_data(person, 'beat', 'rhythm', sampling_rate*3)

In [ ]:
def rhythm_model(re_X_train_r):
    input_rhythm = Input(shape=(re_X_train_r,1), name = 'input_rhythm')

    conv1_R = Conv1D(8, 5, activation=None, padding='same', kernel_initializer=weight_init)(input_rhythm)
    conv2_R = Conv1D(8, 5, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R)
    bn1_R = BatchNormalization()(conv2_R)
    act_R = Activation('relu')(bn1_R)
    max1_R = MaxPooling1D(2,2)(act_R)

    conv1_R_1 = Conv1D(16, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_R)
    conv2_R_1 = Conv1D(16, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R_1)
    bn1_R_1 = BatchNormalization()(conv2_R_1)
    act_R_1 = Activation('relu')(bn1_R_1)
    max1_R_1 = MaxPooling1D(2,2)(act_R_1)


    conv1_R_2 = Conv1D(32, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_R_1)
    conv2_R_2 = Conv1D(32, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R_2)
    bn1_R_2 = BatchNormalization()(conv2_R_2)
    act_R_2 = Activation('relu')(bn1_R_2)
    max1_R_2 = MaxPooling1D(2,2)(act_R_2)


    conv1_R_3 = Conv1D(64, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_R_2)
    conv2_R_3 = Conv1D(64, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R_3)
    bn1_R_3 = BatchNormalization()(conv2_R_3)
    act_R_3 = Activation('relu')(bn1_R_3)
    max1_R_3 = MaxPooling1D(2,2)(act_R_3)


    conv1_R_4 = Conv1D(128, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_R_3)
    conv2_R_4 = Conv1D(128, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R_4)
    bn1_R_4 = BatchNormalization()(conv2_R_4)
    act_R_4 = Activation('relu')(bn1_R_4)
    max1_R_4 = MaxPooling1D(2,2)(act_R_4)

    GAP_R = GlobalAveragePooling1D()(max1_R_4)

    re_R = Reshape((1, GAP_R.shape[1]))(GAP_R)
    x_R_LSTM = LSTM(128,return_sequences=True)(re_R)
    x_R_LSTM2 = LSTM(64, return_sequences=True)(x_R_LSTM)
    fl_f = Flatten()(x_R_LSTM2)

    den1 = Dense(128, activation='relu')(fl_f)
    den2 = Dense(64, activation='relu')(den1)

    fusion_output = Dense(4, activation='softmax')(den2)

    fusion_model = Model(inputs= input_rhythm, outputs= fusion_output)
    fusion_model.summary()
    
    return fusion_model

In [ ]:
## layer 실험
number = 1
save_model_path = "D:/Journal/WESAD_Model/Beat_Rhythm_5Fold_실험/"
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=128)

for train_index, test_index in skf.split(train_rhythm, train_y):
    X_train_b, X_test_b = train_rhythm[train_index], train_rhythm[test_index]
    Y_train, Y_test = train_y[train_index], train_y[test_index]

    y_train = to_categorical(Y_train)
    y_test = to_categorical(Y_test)
    class_weight = cal_class_weight(Y_train)

    MAUS_beat_model = rhythm_model(X_train_b.shape[1])
    callback_lr = tf.keras.callbacks.LearningRateScheduler(scheuler)
    MAUS_beat_model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    history = MAUS_beat_model.fit(X_train_b, y_train, 
                        validation_data=(X_test_b, y_test), 
                        epochs=50, 
                        batch_size=32,
                        callbacks=[callback_lr],
                        class_weight=class_weight
                       ) 
    
    
    scores = MAUS_beat_model.evaluate(X_test_b, y_test)
    Plot_lr_curve(history)
    pred = MAUS_beat_model.predict(X_test_b)
    Plot_confusion_matrix(y_test, pred, class_name)
    
    
    MAUS_beat_model.save(save_model_path + 'WESAD_rhythm_8020_5_layer_(0.0001)_Fold_'  +str(number)+ '(' + str(np.round(scores[1]*100,2)) + ').h5')


# 6 layer and 8 second

In [ ]:
train_beat, train_rhythm, train_y = load_data(person, 'beat', 'rhythm', sampling_rate*8)

In [ ]:
def rhythm_model(re_X_train_r):
    input_rhythm = Input(shape=(re_X_train_r,1), name = 'input_rhythm')

    conv1_R = Conv1D(8, 5, activation=None, padding='same', kernel_initializer=weight_init)(input_rhythm)
    conv2_R = Conv1D(8, 5, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R)
    bn1_R = BatchNormalization()(conv2_R)
    act_R = Activation('relu')(bn1_R)
    max1_R = MaxPooling1D(2,2)(act_R)

    conv1_R_1 = Conv1D(16, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_R)
    conv2_R_1 = Conv1D(16, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R_1)
    bn1_R_1 = BatchNormalization()(conv2_R_1)
    act_R_1 = Activation('relu')(bn1_R_1)
    max1_R_1 = MaxPooling1D(2,2)(act_R_1)


    conv1_R_2 = Conv1D(32, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_R_1)
    conv2_R_2 = Conv1D(32, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R_2)
    bn1_R_2 = BatchNormalization()(conv2_R_2)
    act_R_2 = Activation('relu')(bn1_R_2)
    max1_R_2 = MaxPooling1D(2,2)(act_R_2)


    conv1_R_3 = Conv1D(64, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_R_2)
    conv2_R_3 = Conv1D(64, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R_3)
    bn1_R_3 = BatchNormalization()(conv2_R_3)
    act_R_3 = Activation('relu')(bn1_R_3)
    max1_R_3 = MaxPooling1D(2,2)(act_R_3)


    conv1_R_4 = Conv1D(128, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_R_3)
    conv2_R_4 = Conv1D(128, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R_4)
    bn1_R_4 = BatchNormalization()(conv2_R_4)
    act_R_4 = Activation('relu')(bn1_R_4)
    max1_R_4 = MaxPooling1D(2,2)(act_R_4)

    conv1_R_5 = Conv1D(256, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_R_4)
    conv2_R_5 = Conv1D(256, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R_5)
    bn1_R_5 = BatchNormalization()(conv2_R_5)
    act_R_5 = Activation('relu')(bn1_R_5)
    max1_R_5 = MaxPooling1D(2,2)(act_R_5)

    GAP_R = GlobalAveragePooling1D()(max1_R_5)

    re_R = Reshape((1, GAP_R.shape[1]))(GAP_R)
    x_R_LSTM = LSTM(128,return_sequences=True)(re_R)
    x_R_LSTM2 = LSTM(64, return_sequences=True)(x_R_LSTM)
    fl_f = Flatten()(x_R_LSTM2)

    den1 = Dense(128, activation='relu')(fl_f)
    den2 = Dense(64, activation='relu')(den1)

    fusion_output = Dense(4, activation='softmax')(den2)

    fusion_model = Model(inputs= input_rhythm, outputs= fusion_output)
    fusion_model.summary()
    
    return fusion_model

In [ ]:
## layer 실험
number = 1
save_model_path = "D:/Journal/WESAD_Model/Beat_Rhythm_5Fold_실험/"
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=128)

for train_index, test_index in skf.split(train_rhythm, train_y):
    X_train_b, X_test_b = train_rhythm[train_index], train_rhythm[test_index]
    Y_train, Y_test = train_y[train_index], train_y[test_index]

    y_train = to_categorical(Y_train)
    y_test = to_categorical(Y_test)
    class_weight = cal_class_weight(Y_train)

    MAUS_beat_model = rhythm_model(X_train_b.shape[1])
    callback_lr = tf.keras.callbacks.LearningRateScheduler(scheuler)
    MAUS_beat_model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    history = MAUS_beat_model.fit(X_train_b, y_train, 
                        validation_data=(X_test_b, y_test), 
                        epochs=50, 
                        batch_size=32,
                        callbacks=[callback_lr],
                        class_weight=class_weight
                       ) 
    
    
    scores = MAUS_beat_model.evaluate(X_test_b, y_test)
    Plot_lr_curve(history)
    pred = MAUS_beat_model.predict(X_test_b)
    Plot_confusion_matrix(y_test, pred, class_name)
    
    
    MAUS_beat_model.save(save_model_path + 'WESAD_beat_8020_6_layer_(0.0001)_Fold_'  +str(number)+ '(' + str(np.round(scores[1]*100,2)) + ').h5')
